In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def preprocess(data_path):
    dataset = pd.read_csv(data_path)
    X = dataset.iloc[:, 3:13].values
    y = dataset.iloc[:, 13].values
    labelencoder_X_1 = LabelEncoder()
    X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
    labelencoder_X_2 = LabelEncoder()
    X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
    onehotencoder = OneHotEncoder(categorical_features = [1])
    X = onehotencoder.fit_transform(X).toarray()
    X = X[:, 1:]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, y_train, X_test, y_test

In [3]:
X_train, y_train, X_test, y_test = preprocess('datasets/Churn_Modelling.csv')

In [276]:
units_per_layer=[11,6,1]
kernel_init='uniform'
activation=['relu','relu', 'sigmoid']

In [2]:
%%writefile neural_net.py
from collections import OrderedDict
from scipy.special import expit
import numpy as np

class NeuralNetwork:
    """
    NeuralNetwork:{
    layer_id:{
                weights: np.array()
        }
    }
    
    Example:
    NeuralNetwork:{
    1:{
        'weights':[0.1,-0.54,0.32]
        }
    2:{
        'weights':[0.31,-0.344,0.41, 0.89]
        }    
    }
    
    """
    def __init__(self, units_per_layer, kernel_init, activation):
        self.units_per_layer = units_per_layer
        self.kernel_init = kernel_init
        self.activation = activation
        self.layers = OrderedDict()
        
        # initialize weights and biases 
        for layer_id, units in enumerate(self.units_per_layer[:-1]):
            prev_dims = self.units_per_layer[layer_id]
            num_units = self.units_per_layer[layer_id+1]
            layer_dims = (num_units, prev_dims)
            #layer_dims = (prev_dims, num_units, self.batch_size)
            
            self.layers[layer_id+1] = {}
            self.layers[layer_id+1]['weights'] = np.random.uniform(-1,1, layer_dims)
            # we can add biases later if we want
    
    def forward_pass(self, ipt_matrix):
        
        for layer_id, layer in self.layers.items():
            #print('layer number ->', layer_id)
            V = np.dot((ipt_matrix), (layer['weights'].T))
                
#                 print('multiplied matrices -> \n weights -> {}, \
#                 ipt_matrix ->{}'.format(layer['weights'].T.shape, ipt_matrix.shape))
#                 print('output matrix shape ', V.shape)
            y = self.activation_function(V,activation_type=self.activation[int(layer_id)])
            ipt_matrix = y
            
        return y
    
    
    @staticmethod
    def activation_function(V, activation_type='relu'):
        if activation_type == 'relu':
            return np.maximum(V, 0, V)
        elif activation_type == 'sigmoid':
            return expit(V)
        else:
            raise('activation type {} not found!'.format(activation_type))
            

Overwriting neural_net.py


In [298]:
classifier = NeuralNetwork(units_per_layer, kernel_init, activation, batch_size)

In [299]:
print('shape of layer 1 ',classifier.layers[1]['weights'].shape)
print('shape of layer 2 ',classifier.layers[2]['weights'].shape)

shape of layer 1  (6, 11)
shape of layer 2  (1, 6)


In [300]:
# lets try to randomly sample a batch of batch_size
sample_idxs = np.random.choice(X_train.shape[0], batch_size, replace=False)
sample_training_batch = X_train[sample_idxs, :]
y_train = y_train.reshape(y_train.shape[0], 1)
sample_training_answer = y_train[sample_idxs, :]
sample_training_batch.shape
print(X_train.shape, 'id to sample: ', sample_idxs, 'shape of y_train', y_train.shape)

(8000, 11) id to sample:  [6252 3658 1243 5052] shape of y_train (8000, 1)


In [301]:
classifier.forward_pass(sample_training_batch)

layer number -> 1
multiplied matrices -> 
 weights -> (11, 6),                 ipt_matrix ->(4, 11)
output matrix shape  (4, 6)
layer number -> 2
multiplied matrices -> 
 weights -> (6, 1),                 ipt_matrix ->(4, 6)
output matrix shape  (4, 1)


array([[0.13200311],
       [0.1918486 ],
       [0.78970992],
       [0.541071  ]])